<a href="https://colab.research.google.com/github/felipmorais/felipmorais/blob/main/Calculo_da_Dinamica_de_Afetos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Desenvolvimento do modelo de dinâmica de afetos no contexto Brasileiro**
---


Este código foi desenvolvido para calcular o modelo de dinâmica de afetos. Diferente de outros códigos abertos, este código foi desenvolvido para:

1. Calcular a métrica L ajustada;
2. Aplicar um teste t para calcular a significância estatística dos resultados considerando todos os alunos;
3. Aplicar o cálculo de ajuste do valor p para múltiplos testes, considerando o método de Benjamini/Hochberg (BH);
4. Imprimir as estatísticas calculadas em forma tabular;
5. Renderizar o modelo gráfico de dinâmica de afetos automaticamente com base nas estatísticas calculadas, considerando apenas as transições reportadas como significativas após o ajuste de valor p com base no método BH;



---
**Importante:** Este código é parte de um artigo científico submetido à avaliação. Os autores e afiliações foram removidos para Blind Review. 
*Por favor, não compartilhe esse código até a publicação do artigo.*



Execução 1: Arquivo de cálculo da métrica L (importado de [Matayoshi and Karumbaiah 2020]
---
Este primeiro código é importado do artigo "Adjusting the L Statistic when Self-Transitions are Excluded in Affect Dynamics", dos autores Jeffrey Matayoshi e Shamya Karumbaiah, 2020. 

O código foi mantido por completo para simplificação. A única alteração foi feita no método 'compute_statistic', que retornava apenas a matrix com a média dos valores L. O ajuste foi para trazer a lista de todos os valores L para a aplicação de teste t, para o cálculo do valor p.


In [1]:
# código clonado de  https://github.com/jmatayoshi/affect-transitions

import numpy as np

def remove_self_transitions(seq):
    return [st for i, st in enumerate(seq) if i == 0 or st != seq[i - 1]]


def generate_sequence(seq_length, state_dict, base_rates,
                      include_self_transitions=True):
    state_ind = np.arange(len(state_dict))
    if include_self_transitions:
        seq = list(
            np.random.choice(
                list(state_dict.values()),
                size=seq_length,
                p=base_rates
            )
        )
    else:
        base_rates = np.array(base_rates)
        # Construct mapping from each state to its index in state_dict
        inv_state_dict = {}
        for i in range(len(state_dict)):
            inv_state_dict[state_dict[i]] = i
        # Sample first state
        seq = [state_dict[np.random.multinomial(1, base_rates).argmax()]] 
        for i in range(seq_length - 1):
            # Get index of previous state
            prev_ind = inv_state_dict[seq[-1]]
            # Get indices of possible next states
            next_ind = np.setdiff1d(state_ind, [prev_ind])
            # Construct conditional rates of next states
            cond_next_rates = base_rates[next_ind] / (1 - base_rates[prev_ind])
            # Sample next state
            # np.random.multinomial is faster here than np.random.choice
            seq.append(state_dict[next_ind[
                np.random.multinomial(1, cond_next_rates).argmax()]])
    return seq


def get_counts(seq, states):
    next_count = {a: 0 for a in states}
    cond_count = {a: {b: 0 for b in states} for a in states}
    num_tr = len(seq) - 1
    # Compute next and conditional counts
    for i in np.arange(1, len(seq)):
        for a in states:
            if seq[i - 1] == a:
                for b in states:
                    if seq[i] == b:
                        cond_count[a][b] += 1
                        next_count[b] += 1
                        break
                break

    cond_count_list = []
    for a in cond_count:
        cond_count_list.extend(list(cond_count[a].values()))

    return list(next_count.values()), cond_count_list


def get_L_star_vals(a, b, next_counts, cond_counts, use_mean_rates=True):
    num_states = next_counts.shape[1]
    # Column indices where next != a (i.e., transitions in T_{A_complement})
    a_comp_ind = (
        np.array([i for i in range(num_states) if i != a])
    )
    # Count transitions where prev == a and next != a
    a_comp_cond_sum = cond_counts[:, a_comp_ind + a*num_states].sum(axis=1)
    if use_mean_rates:      
        # Compute L_star using base rates averaged over the whole sample
        # of sequences; note that as opposed to the computation of
        # L_star below, we only exclude samples with P(b|a) == nan; that is,
        # we only exclude sequences with no transitions from a to another state
        sample_pos = np.flatnonzero(
            a_comp_cond_sum > 0
        )
        # Compute mean base rate of b restricted to transitions with next != a
        modified_mean_base_rate = np.mean(
            next_counts[sample_pos, b] /
            next_counts[sample_pos, :][:, a_comp_ind].sum(axis=1)
        )
        # Compute conditional rate of b restricted to transitions with next != a
        cond_rates = (
            cond_counts[sample_pos, a*num_states + b] /
            a_comp_cond_sum[sample_pos]
        )
        L_star_vals = (
            (cond_rates - modified_mean_base_rate)
            / (1 - modified_mean_base_rate)
        )
    else:
        # Compute L_star using base rates from each individual sequence

        # Column indices where next != a and next != b
        a_b_comp_ind = (
            np.array([i for i in range(num_states) if i != a and i != b])
        )
        # Count transitions where next != a or next != b
        a_b_comp_sum = next_counts[:, a_b_comp_ind].sum(axis=1)
        # Count transitions where next != a        
        a_comp_sum = next_counts[:, b] + a_b_comp_sum
        # Find samples where:
        #  (a) P(b|a) != nan
        #  (b) P(b) < 1
        sample_pos = np.flatnonzero(
            (a_comp_cond_sum > 0) & (a_b_comp_sum > 0)            
        )        
        # Compute base rates of b restricted to transitions with next != a
        modified_base = (
            next_counts[sample_pos, b] / a_comp_sum[sample_pos]
        )
        # Compute conditional rate of b restricted to transitions with next != a
        cond_rates = (
            cond_counts[sample_pos, a*num_states + b] /
            a_comp_cond_sum[sample_pos]
        )       
        L_star_vals = (
            (cond_rates - modified_base)
            / (1 - modified_base)
        )
    return L_star_vals

def get_L_vals(a, b, next_counts, cond_counts, use_mean_rates=True):
    num_states = next_counts.shape[1]
    # Count transitions where prev == a and next != a
    a_cond_sum = cond_counts[
        :, np.arange(num_states) + a*num_states].sum(axis=1)
    if use_mean_rates:      
        # Compute L using base rates averaged over the whole sample
        # of sequences.  Note that as opposed to the computation of
        # L below, we only exclude samples with P(b|a) == nan; that is,
        # we only exclude sequences with no transitions from a 
        sample_pos = np.flatnonzero(
            a_cond_sum > 0
        )
        # Compute mean base rate of b 
        mean_base_rate = np.mean(
            next_counts[sample_pos, b] /
            next_counts[sample_pos, :].sum(axis=1)
        )
        # Compute conditional rate of b 
        cond_rates = (
            cond_counts[sample_pos, a*num_states + b] /
            a_cond_sum[sample_pos]
        )
        L_vals = (
            (cond_rates - mean_base_rate)
            / (1 - mean_base_rate)
        )
    else:
        # Compute L using base rates from each individual sequence

        # Column indices where next != a and next != b
        b_comp_ind = (
            np.array([i for i in range(num_states) if i != b])
        )
        # Count transitions where next != b
        b_comp_sum = next_counts[:, b_comp_ind].sum(axis=1)
        # Find samples where:
        #  (a) P(b|a) != nan
        #  (b) P(b) < 1
        sample_pos = np.flatnonzero(
            (a_cond_sum > 0) & (b_comp_sum > 0)            
        )        
        # Compute base rates of b
        base_rates = (
            next_counts[sample_pos, b] / next_counts[sample_pos, :].sum(axis=1)
        )
        # Compute conditional rate of b
        cond_rates = (
            cond_counts[sample_pos, a*num_states + b] /
            a_cond_sum[sample_pos]
        )       
        L_vals = (
            (cond_rates - base_rates)
            / (1 - base_rates)
        )
    return L_vals


def compile_sequence_counts(seq_list, states):
    next_counts = []
    cond_counts = []    
    for seq in seq_list:
        count_res = get_counts(seq, states)
        next_counts.append(count_res[0])
        cond_counts.append(count_res[1])
    return np.array(next_counts), np.array(cond_counts)


def print_vals(val_array, state_dict, title):
    print('\n\n' + title + '\n')
    print('Prev\\Next' + '\t' + '\t'.join(map(str, list(state_dict.values()))))
    for i in range(len(state_dict)):
        print(state_dict[i] + '\t\t'
              + '\t'.join(map(str, val_array[i, :].round(4))))
    return


def run_simulations(
        num_trials=50000,
        seq_length=21,
        states=['A', 'B', 'C', 'D'],
        base_rates=np.array([0.45, 0.45, 0.05, 0.05]),
        verbose=True,
        include_self_transitions=True,
        compute_L=True
):
    num_states = len(states)
    state_dict = {}
    for i in range(num_states):
        state_dict[i] = states[i]
    seq_list = []
    reduced_seq_list = []
    for i in range(num_trials):
        seq = generate_sequence(seq_length, state_dict, base_rates,
                    include_self_transitions=include_self_transitions)        
        reduced_seq = remove_self_transitions(seq)

        seq_list.append(seq)
        reduced_seq_list.append(reduced_seq)

    res = []
    # Compute L_star using individual base rates from full sequences with
    # self-transitions included     
    res.append(
        compute_statistic(seq_list, states, L_star=True,
                          use_mean_rates=False)
    )
    # Compute L_star using mean base rates from full sequences with
    # self-transitions included         
    res.append(
        compute_statistic(seq_list, states, L_star=True,
                          use_mean_rates=True)
    )
    if compute_L:
        # Compute L using individual base rates from reduced sequences with
        # self-transitions removed             
        res.append(
            compute_statistic(seq_list, states, L_star=False,
                              use_mean_rates=False)
        )
        # Compute L_star using mean base rates from full sequences with
        # self-transitions included                  
        res.append(
            compute_statistic(seq_list, states, L_star=False,
                              use_mean_rates=True)
        )            

    
    if verbose:
        title_list = [
            'L_star with individual base rates from full sequences',
            'L_star with mean base rates from full sequences',
            'L with individual base rates from reduced sequences',
            'L with mean base rates from reduced sequences',
        ]
        end_ind = 2
        if compute_L:
            end_ind = len(title_list)
        for i in range(end_ind):
            print_vals(res[i], state_dict, title_list[i])
            
    return (
        seq_list,
        reduced_seq_list,
        res
    )


def compute_statistic(seq_list, states, L_star=True,
                      use_mean_rates=True):
    """ General function for computing L_star and L statistics

    Parameters
    ----------
    seq_list : list of lists
        Each entry in the list is a sequence (list) of affective states; note 
        that self-transitions are automatically removed if L_star is false
        Example:
            [
                ['A', 'C', 'C', 'B', 'C'],
                ['B', 'C', 'A', 'C'],
                ['C', 'C', 'C', 'B', 'B', 'A']
            ]
    states : list 
        List containing all possible affective states 
        Example:
            ['A', 'B', 'C']
    L_star : bool, default=True 
        If true compute L_star statistic; otherwise, remove 
        self-transitions and compute L statistic
    use_mean_rates : bool, default=True
        If true compute base rates averaged over all sequences; otherwise,
       compute base rates individually per sequence
    """
    if L_star:
        input_list = seq_list
    else:
        input_list = []
        for i in range(len(seq_list)):
            input_list.append(remove_self_transitions(seq_list[i]))
        
    next_counts, cond_counts = compile_sequence_counts(input_list, states)

    num_states = len(states)
    res = np.full((num_states, num_states), np.nan)
    test = np.full((num_states, num_states), object)
    for i in range(num_states):
        for j in range(num_states):
            if i != j:
                if L_star:
                    aux = get_L_star_vals(i, j, next_counts, cond_counts, use_mean_rates=use_mean_rates)

                    res[i, j] = np.mean(aux)
                    test[i, j] = aux
                else:
                    aux = get_L_vals(i, j, next_counts, cond_counts, use_mean_rates=use_mean_rates)
                    
                    res[i, j] = np.mean(aux)
                    test[i, j] = aux

    return res, test


def base_rate_analysis(
        states=['A', 'B', 'C', 'D'],
        base_rates=np.ones(4)*0.25,
        num_steps=24,
        rate_step=[0.03, -0.01, -0.01, -0.01],
        num_trials=50000,
        seq_length=21
):
    """ Run numerical experiments 
    
    Experiment 1 parameters: 
        states=['A', 'B', 'C', 'D'], 
        base_rates=np.ones(4)*0.25, 
        num_steps=24, 
        rate_step=[0.03, -0.01, -0.01, -0.01],
        num_trials=50000,
        seq_length=21

    Experiment 2 parameters: 
        states=['A', 'B', 'C', 'D'], 
        base_rates=np.ones(4)*0.25, 
        num_steps=23, 
        rate_step=[0.01, 0.01, -0.01, -0.01],
        num_trials=50000,
        seq_length=21
    
    """
    rate_step = np.array(rate_step)
    indiv_rate_results = []
    mean_rate_results = []
    all_base_rates = []
    for i in range(num_steps):
        sim_res = run_simulations(
            num_trials=num_trials,
            seq_length=seq_length,
            states=states,
            base_rates=base_rates,
            verbose=False,
            include_self_transitions=True,
            compute_L=False
        )
        indiv_rate_results.append(sim_res[2][0])
        mean_rate_results.append(sim_res[2][1])
        all_base_rates.append(list(base_rates))
        if i < num_steps - 1:
            base_rates += rate_step
    return indiv_rate_results, mean_rate_results, all_base_rates



Execução 2: Carga do arquivo de rótulos de emoções
---
Este código é utilizado para fazer a importação e pré-processamento do arquivo de emoções. É importante que esse arquivo esteja ordenado de acordo com a ocorrencia das emoções e com o id do respectivo aluno.


In [ ]:
# importacao das libs utilizadas
# import numpy as np
import pandas as pd 
from scipy import stats
import statsmodels.stats.multitest as multi
from matplotlib import pyplot as plt
# %matplotlib inline
import networkx as nx

from google.colab import drive

# le o arquivo de dentro do google drive
drive.mount('/content/drive')


# arquivo com todas as emoções e com o id do aluno, no seguinte padrão:
# 
# student_id,emotiona_label
# 1,ENGAJAMENTO
# 1,ENGAJAMENTO
# 1,CONFUSAO
# 1,CONFUSAO
# 1,FRUSTRACAO
# 2,TEDIO
# 2,TEDIO
# 2,OUTRO
# 3,CONFUSAO
# ....
# 
emotion_labels_path = "/content/drive/My Drive/<INFORMAR_PATH_DO_ARQUIVO_DE_ROTULOS_DE_EMOCOES>/emotion_labels.csv" # IMPORTANTE: colocar aqui o path correto para o arquivo a ser importado com os rotulos das emocoes dos alunos

# le o arquivo passado no path indicado e salva o resultado em um data fram, utilizando a lib pandas
df_emotion_labels = pd.read_csv(emotion_labels_path, delimiter=",")


# abrevia os labels de emocoes para a geracao do grafo
df_emotion_labels.loc[df_emotion_labels.emotion_label == "FRUSTRACAO", "emotion_label"] = "FRU"
df_emotion_labels.loc[df_emotion_labels.emotion_label == "ENGAJAMENTO", "emotion_label"] = "ENG"
df_emotion_labels.loc[df_emotion_labels.emotion_label == "CONFUSAO", "emotion_label"] = "CON"
df_emotion_labels.loc[df_emotion_labels.emotion_label == "TEDIO", "emotion_label"] = "TED"
df_emotion_labels.loc[df_emotion_labels.emotion_label == "OUTRO", "emotion_label"] = "OUT"

# remove linhas com label de emocao null
df_emotion_labels = df_emotion_labels[df_emotion_labels.emotion_label.notna()]



Execução 3: Definição das funções de cálculo da métrica L e estatística
---
Esta parte do código é a definição das funções auxiliares para o cálculo da métrica L, das estatísticas de valor p e multiplos testes e para a parte de impressão dos resultados e do gráfico de dinâmica de afetos.


In [6]:

# Função que computa a lista de alunos que devem ser incluidos na analise da dinâmica de emoções
# retorna uma lista com os ids dos alunos que serão incluidos
#   Exemplo de retorno: [1, 2, 3, 4, 5]

# recebe o número de alunos que deve ser considerados -> students_amount
#   por default esse valor está 30, mas pode ser passado qualquer valor por parametro

# recebe também um boleano que informa se é para exibir as informações dos alunos selecionados -> show_info
#   por default esse valor é True

def compute_students_to_include(students_amount=30, 
                                show_info=True):

  students_to_include = []
  for n_student in range(1, students_amount + 1, 1):
    include_n_student = True

    # incluir alguma condição para a exclusão de um determinado aluno
    
    if include_n_student:
      students_to_include.append(n_student)

  if show_info:
    print("Número de alunos: ", len(students_to_include))
    print("Ids dos alunos inclusos: ", students_to_include)
  
  return students_to_include


# receives a list of students ids to be considered
# and a boolean whether the emotion duration should be considered or not

# Função que recebe uma lista de ids de alunos para ser incluidos na análise e retorna a lista de rótulos de emoções para os alunos inclusos
# Retorna uma lista de listas, onde cada lista representa as emoções de um aluno
# Exemplo de retorno
#   [[ENGAJAMENTO,ENGAJAMENTO,FRUSTRACAO,TEDIO], [CONFUSAO,CONFUSAO,OUTRA,OUTRA,FRUSTRACAO], ...]
# Retorna também a lista de emoções consideradas (abreviadas)

# recebe a lista de ids de alunos a serem incluidos em forma de array -> students_to_include
#   por default esse valor é um array vazio [], mas deve ser passado um array com os ids de alunos a serem analisados

# recebe também um boleano que informa se é para exibir as informações dos alunos selecionados -> show_info
#   por default esse valor é True

def compute_list_of_emotion_labels(students_to_include=[],
                                   show_info=True):

  # computa as sequencias de emocoes por aluno
  transitions_list = []
  for n_student in students_to_include:
    transitions_list.append(list(df_emotion_labels[df_emotion_labels.student_id == n_student].emotion_label.values))

  # lista de estados afetivos
  # poderia pegar dinamicamente, mas preferimos deixar fixo para que as tabelas e graficos siguam sempre o mesmo padrão
  states_list = ['ENG', 'CON', 'FRU', 'TED', 'OUT'] 

  if show_info:
    print("Estados Afetivos: ", states_list)
    print("Número de listas (número de alunos analisados): ", len(transitions_list))
    print("Número de rótulos de cada lista:")
    for i in range(len(transitions_list)):
      print("Id do aluno -> " + str(students_to_include[i]) + ": " + str(len(transitions_list[i])))

  return states_list, transitions_list


# Computa o número de transições entre todos os estados afetivos e alunos
# Recebe a lista de listas de emoções dos alunos, produzidas em compute_list_of_emotion_labels
# Recebe a lista de estados afetivos contemplados
# Retorna uma matrix com a quantidade de transições considerando todos os estados para todos os estados
def get_emotion_transitions_counts(list_seq, states):
  cond_count = {a: {b: 0 for b in states} for a in states}
  for j in np.arange(0, len(list_seq)):
    seq = list_seq[j]
    num_tr = len(seq) - 1
    # Compute next and conditional counts
    for i in np.arange(1, len(seq)):
      for a in states:
        if seq[i - 1] == a:
          for b in states:
            if seq[i] == b:
              cond_count[a][b] += 1
              break
          break

  return cond_count


# Recebe uma matrix com a média dos valores da métrica L (considerando todos os alunos) para cada combinação de  transição entre emoções e calcula o valor p para cada combinação
# Recebe a lista de emoções consideradas (abreviadas), a matrix L calculada com base na média dos alunos e o valor ao acaso (essa parte é o valor L ajustado de acordo com Karumbaiah e colegas).
# Como este exemplo utiliza 5 emoções, por padrão o valor está como 0.0625
# Retorna uma matrix com o valor p para cada combinação de transição entre emoções
# Para o calculo dos valores p, é aplicado um teste t para as médias de um grupo de valores
#   para mais informações sobre o teste t, ver: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html
def calculate_p_values(states_list, L_metric_average_results, level_at_change=0.0625):
  num_states = len(states_list)
  matrix_p_values = np.full((num_states, num_states), np.nan)
  for i in range(num_states):
    for j in range(num_states):
      if i != j:
        matrix_p_values[i][j] = stats.ttest_1samp(list(L_metric_average_results[i][j]), level_at_change).pvalue
  return matrix_p_values

# Função que calcula o ajuste de valor p para multiplos testes estatísticos.
# Recebe a lista de emoções consideradas (abreviado) e a matrix com os valores p
# Para o cálculo de ajuste de valor p para multiplos testes é aplicado o método Benjamini/Hochberg.
#   para mais informações sobre o método Benjamini/Hochberg, ver: https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.multipletests.html
# retorna uma matrix com os valores p ajustados e outra matrix com os valores p que devem ser rejeitados
# lembrando que essa matrix sempre representa todas as combinações de transições entre todos as emoções
def calculate_multiple_t_test_adjustment(states_list, matrix_p_values):
  num_states = len(states_list)
  multiple_tests_matrix = np.full((num_states ,num_states), np.nan)
  multiple_tests_reject_matrix = np.full((num_states ,num_states), np.nan)

  for i in range(num_states):
    pvals =[x for x in matrix_p_values[i] if str(x) != 'nan']
    if len(pvals) > 0:
      line_by = multi.multipletests(pvals, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)
      for j in range(num_states - 1):
        column = j
        if j >= i:
          column = column + 1
        multiple_tests_matrix[i][column] = line_by[1][j]
        multiple_tests_reject_matrix[i][column] = line_by[0][j]
  
  return multiple_tests_matrix, multiple_tests_reject_matrix


# Função para imprimir a matrix de transição entre emoções com os valores calculados
# Esta função é configurável, proporcionando a impressão com a informação que deseja ser exibida.
# Para informar quais valores deverão ser impressos, basta passar os valores para o array de strings chamado print_types, os valores aceitos são:
#   L_metric
#   p_value
#   multiple_test_p_value
#   transition_count
#   multiple_test_significance

# imprime a matrix separada por ; assim é só copiar e colar no excel
def print_vals(l_matrix, p_value_matrix, multiple_tests_matrix, multiple_tests_reject_matrix, count_matrix, states_list, print_types):

  print('Prev\\Next' + ';' + ';'.join(map(str, states_list)))
  for i in range(len(states_list)):
    print(states_list[i] + ';', end ="")
    for j in range(len(states_list)):
      if "L_metric" in print_types:
        print(format((np.zeros(1)[0] if np.isnan(l_matrix[i,j]) else l_matrix[i,j]).round(4), '.4f'), end="", flush=True)
      if "p_value" in print_types:
        print(' (', end="", flush=True) if len(print_types) > 1 else False
        print(format((np.zeros(1)[0] if np.isnan(p_value_matrix[i,j]) else p_value_matrix[i,j]).round(4), '.4f'), end="", flush=True)
        print(') ', end="", flush=True) if len(print_types) > 1 else False
      if "multiple_test_p_value" in print_types:
        print(' |', end="", flush=True) if len(print_types) > 1 else False
        print(format((np.zeros(1)[0] if np.isnan(multiple_tests_matrix[i,j]) else multiple_tests_matrix[i,j]).round(4), '.4f'), end="", flush=True)
        print('| ', end="", flush=True) if len(print_types) > 1 else False
      if "transition_count" in print_types:
        print(' [', end="", flush=True) if len(print_types) > 1 else False
        print(str((np.zeros(1)[0] if np.isnan(count_matrix[states_list[i]][states_list[j]]) else count_matrix[states_list[i]][states_list[j]])), end="", flush=True) 
        print('] ', end="", flush=True) if len(print_types) > 1 else False
      if "multiple_test_significance" in print_types:
        print(' {', end="", flush=True) if len(print_types) > 1 else False
        print(format((np.zeros(1)[0] if np.isnan(multiple_tests_reject_matrix[i,j]) else multiple_tests_reject_matrix[i,j]).round(1), '.1f'), end="", flush=True)
        print('} ', end="", flush=True) if len(print_types) > 1 else False
      
      if j < len(states_list) - 1:
        print(';', end="", flush=True)
    print("")
  return

# Função que gera o grafo de dinâmica de afetos de acordo com a estatística calculada
# Esta função recebe a matrix com os valores L, a matrix de rejeição de valores p de acordo com o ajuste de multiplos testes, a lista de emoções consideradas e o valor L ao acaso
def print_affect_dynamics_graph(l_matrix, multiple_tests_reject_matrix, states_list, threshold=0.0625):
  fig, ax = plt.subplots(figsize=(10,10))

  G = nx.MultiDiGraph()  # Create empty graph

  # create the nodes based on the states list
  G.add_nodes_from(states_list)

  for i in range(len(states_list)):
    for j in range(len(states_list)):
      is_significant = (np.zeros(1)[0] if np.isnan(multiple_tests_reject_matrix[i,j]) else multiple_tests_reject_matrix[i,j]).round(1)
      if i != j and is_significant > 0:
        transition_l = (np.zeros(1)[0] if np.isnan(l_matrix[i,j]) else l_matrix[i,j]).round(2)
        transition_color = 'red' if transition_l < threshold else 'blue'
        # add the edges with the l_metric as weight only if the transition is significant (based on the multiple test adjustment)
        # change the color for red if the transition is unlikely and blue if it is more likely to occur
        G.add_edge(states_list[i], states_list[j], color=transition_color, weight=format(transition_l, '.2f'))
  
  pos =  nx.circular_layout(G, scale=1)  # List of positions of nodes

  colors = []

  for (u,v,attrib_dict) in list(G.edges.data()):
      colors.append(attrib_dict['color'])

  weights = [(0.4 + abs(float(d['weight'])) + abs(float(d['weight'])) * 4) for u,v,d in G.edges(data=True)]

  nx.draw(G, pos, with_labels=True, connectionstyle='arc3, rad = 0.15', node_size=1200, edge_color=colors, width=weights, node_color='lightgreen')
  edge_labels = dict([((u,v,),d['weight']) for u,v,d in G.edges(data=True)])

  nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, label_pos=0.2, font_size=11)




Execução 4: Chamada das funções que calculam e imprimem a dinâmica de afetos
---
Esta última parte é onde as funções para cálculo da métrica L e das métricas estatísticas são executadas.


In [ ]:

# calcula a lista de alunos a serem incluidos na analise
lista_de_ids_de_alunos_considerados = compute_students_to_include(students_amount=30,
                                                       show_info=True)

# computa a lista de emoçoes consideradas e a lista de transição das emoções por aluno
lista_emocoes_consideradas_abreviadas, lista_transicao_emocoes_alunos = compute_list_of_emotion_labels(students_to_include=lista_de_ids_de_alunos_considerados,
                                                               show_info=True)

# chama a função do código que calcula a matrix de valores L e a média dos valores L para cada aluno/transição
matrix_resultante_metrica_L, matrix_media_valores_metrica_L = compute_statistic(seq_list=lista_transicao_emocoes_alunos, 
                                                                                states=lista_emocoes_consideradas_abreviadas, 
                                                                                L_star=False, 
                                                                                use_mean_rates=False)

# calcula o valor p para os valores L reportados
matrix_valores_p = calculate_p_values(states_list=lista_emocoes_consideradas_abreviadas, 
                                      L_metric_average_results=matrix_media_valores_metrica_L, 
                                      level_at_change=0.0625)

# realiza o ajuste do valor p de acordo com o método BH e retorna a matrix que indica quais valores devem ser rejeitados
matrix_valores_p_ajustados_para_multiplos_teste, matrix_rejeicao_multiplos_testes = calculate_multiple_t_test_adjustment(states_list=lista_emocoes_consideradas_abreviadas, 
                                                                                                                         matrix_p_values=matrix_valores_p)

# obtem a informação da quantidade de cada uma das transições entre todas as possiveis combinações de estados afetivos
matrix_contador_transicoes_emocoes = get_emotion_transitions_counts(list_seq=lista_transicao_emocoes_alunos, 
                                                                    states=lista_emocoes_consideradas_abreviadas)

# imprime uma matrix com as informações estatísticas da dinâmica de afetos
#   esta matrix pode ser copiada e colada em um excel, pois utiliza o marcador ; para delimitar as colunas
print_vals(matrix_resultante_metrica_L, 
           matrix_valores_p, 
           matrix_valores_p_ajustados_para_multiplos_teste, 
           matrix_rejeicao_multiplos_testes, 
           matrix_contador_transicoes_emocoes, 
           lista_emocoes_consideradas_abreviadas,
           print_types=['L_metric','p_value','transition_count','multiple_test_significance'])

# gera o gráfico de dinâmica de afetos de acoro com o valor L e com a informação de rejeição de resultados para multiplos testes
print_affect_dynamics_graph(matrix_resultante_metrica_L, matrix_rejeicao_multiplos_testes, lista_emocoes_consideradas_abreviadas, threshold=0.0625)

